![Representative examples of DeepLCMS predictions accompanied by their corresponding probability estimates.](exp-5-prediction_matrix.png){fig-align="center" width=50%}


Welcome to DeepLCMS, a project that combines mass spectrometry analysis with the power of deep learning models!

Unlike traditional methods, DeepLCMS eliminates the need for extensive data processing, including peak alignment, data annotation, quantitation, and other time-consuming steps. Instead, it relies on the power of deep learning to directly classify mass spectrometry-based pseudo-images with high accuracy. To demonstrate the capabilities of pre-trained neural networks for high-resolution LC/MS data, we successfully apply our convolutional neural network (CNN) to categorize substance abuse cases. We utilize the openly available Golestan Cohort Study's metabolomics HRMS/LC data to train and evaluate our CNN [@pourshams_cohort_2010; @ghanbari_metabolomics_2021; @li_untargeted_2020]. We also delve into the network's decision-making process through TorchCam library. This tool allows us to gain insights into the factors that influence the network's classifications, helping us identify key compound classes that play a crucial role in differentiating between classes. By analyzing retention time and molecular weight, we can pinpoint areas of interest within the data.

DeepLCMS paves the way for a new era of mass spectrometry analysis, offering a faster, more efficient, and more insightful approach to data interpretation. Its ability to directly classify pseudo-images without extensive preprocessing opens up a world of possibilities for researchers and clinicians alike.

::: {.callout-note}
**At a glance**

The DeepLCMS project aims to provide researchers with a reproducible source code for leveraging deep learning for mass spectrometry data analysis. It distinguishes itself from previous studies by:
*Comparing Diverse* Architecture Families: Assessing a broader range of architecture families to find the most suitable one, including cutting-edge architectures like vision transformers.
*Hyperparameter Tuning*: Conducting basic hyperparameter tuning to optimize the learning rate using Optuna including optimizer, and learning rate scheduler – crucial aspects beyond the architecture itself.
*Image Quality Analysis*: Investigating the impact of image quality on validation metrics, examining image sharpness and data augmentation imitating retention time shift.
*Regularization Techniques*: Employing regularization techniques like random-tilting images and random erasing during training to improve model generalization.
*Interpreting Pretrained Network Decisions*: Analyzing how the pre-trained network makes its decisions using TorchVision.
:::

# Import libraries

In [ ]:
import io
import re
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from PIL import Image

# Introduction

While computer vision has gained widespread adoption in various aspects of our lives[@dobson_birth_2023], its application in medical imaging and biosciences has lagged behind, primarily due to limitations in clinical dataset size, accessibility, privacy concerns, experimental complexity, and high acquisition costs. For such applications, transfer learning has emerged as a potential solution[@seddiki_towards_2020]. This technique is particularly effective with small datasets, requiring fewer computational resources while achieving good classification accuracy compared to models trained from scratch. Transfer learning involves a two-step process. Initially, a robust data representation is learned by training a model on a dataset comprising a vast amount of annotated data encompassing numerous categories (ImageNet for example). This representation is then utilized to construct a new model based on a smaller annotated dataset containing fewer categories. 

## Application of Pretrained Neural Networks for Mass Spectrometry Data

The use of pre-trained neural networks for mass spectrometry data analysis is relatively new, with only a handful of publications available to date. These studies have demonstrated the potential of deep learning models to extract meaningful information from raw mass spectrometry data and perform predictive tasks without the need for extensive data processing as required by the traditional workflows.

## Previous Research

* In 2018, @behrmann_deep_2018 used deep learning techniques for tumor classification in Imaging Mass Spectrometry (IMS) data.

* In 2020, @seddiki_towards_2020 utilized MALDI-TOF images of rat brain samples to assess the ability of three different CNN architectures – LeNet, Lecun, and VGG9 – to differentiate between different types of cancers based on their molecular profiles.

* In 2021, @cadow_feasibility_2021 explored the use of pre-trained networks for the classification of tumors from normal prostate biopsies derived from SWATH-MS data. They delved into the potential of deep learning models for analyzing raw mass spectrometry data and performing predictive tasks without the need for protein quantification. To process raw MS images, the authors employed pre-trained neural network models to convert them into numerical vectors, enabling further processing. They then compared several classifiers, including logistic regression, support vector machines, and random forests, to accurately predict the phenotype.

* In 2022, @shen_deep_2022 released deepPseudoMSI, a deep learning-based pseudo-mass spectrometry imaging platform, designed to predict the gestational age in pregnant women based on LC-MS-based metabolomics data. This application consists of two components: Pseudo-MS Image Converter: for converting LC-MS data into pseudo-images and the deep learning model itself.


::: {.callout-tip}
## Project Structure

To accommodate the high computational demands of neural network training, the DeepLCMS project is divided into two main parts. The first part focuses on data preprocessing, specifically converting LC/MS data into pseudo-images using the PyOpenMS library, which is written in C++ and optimized for efficiency. This task can be handled on a CPU, and the corresponding source code is found in the `src/deeplcms_functions` directory.

To effectively train the neural networks that demand GPU acceleration, the project employs the PyTorch Lightning framework, a comprehensive solution for building and deploying deep learning models. Training experiments are conducted within Jupyter Notebooks hosted on Google Colab, a cloud platform equipped with free GPU access. The training code and corresponding modules reside within the `src/train_google_colab` directory. This folder seamlessly integrates with Google Colab, allowing for effortless module imports.
:::

# Materials and Methods
## Dataset

To ensure the feasibility of our proof-of-concept demonstration, we selected a suitable dataset from the Metabolomics Workbench. We prioritized studies with distinct groups and a minimum sample size of 200. Additionally, we chose a dataset with a disk requirement of less than 50 GB to minimize computational resource demands. Based on these criteria, we identified the Golestan Cohort Study [@pourshams_cohort_2010]. This study, conducted in northeastern Iran, primarily investigates the risk factors for upper gastrointestinal cancers in this high-risk region. Approximately 50,000 volunteers were analyzed, including opium users and their mortality outcomes. Quantitative targeted liquid chromatography mass spectrometric (LC-MS/MS) data was collected at the University of North Carolina at Chapel Hill [@ghanbari_metabolomics_2021; @li_untargeted_2020]. The dataset consisted of 218 opioid users and 80 non-users. After initial data inspection and conversion to mzML format using the ProteoWizard 3.0.22155 software, files were divided into training (n = 214), validation (n = 54), and test (n = 30) sets. To evaluate the impact of image characteristics and augmentation techniques on classification performance, four datasets were prepared. One dataset employed a bin size of 500 × 500 using the numpy library's histogram2d function and was named `ST001618_Opium_study_LC_MS_500`. Another dataset incorporated data augmentation using the augment_images function, generating nine additional images per training set with random offsets of up to 5 units in both x and y directions. This dataset was named `ST001618_Opium_study_LC_MS_500_augmented`. The third dataset employed a higher bin size of 1000 × 1000 using histogram2d for sharper pseudoimages and was named `ST001618_Opium_study_LC_MS_1000`. The final dataset applied the same augmentation technique to this dataset, generating nine additional images per training set with random offsets of up to 5 units in both x and y directions. It was named `ST001618_Opium_study_LC_MS_1000_augmented`.

## Software

The source code for the DeepLCMS project utilizes the following software packages and versions: PyTorch Lightning 2.1.3, Pytorch Image Models (timm)  0.9.12, torchinfo 1.8.0, Optuna 3.5.0, TorchCam 0.4.0, pandas 2.1.3, NumPy 1.26, and Matplotlib 3.7.1.

## Creating the pseudo images

Mass spectrometry-based pseudo images are generated using the `plot_2D_spectra_overview` function, which creates a 2D heatmap of mass spectrometry data. The function first loads the mass spectrometry data from the provided file path using the OpenMS library. A 2D histogram is created from the data, with the x-axis representing the retention time (RT) and the y-axis representing the m/z values. The intensity values of the histogram are normalized using the highest peak. To enhance the visualization of the data, a Gaussian filter is applied to reduce noise, and the filtered image is scaled to the range 0-255. The histogram is then logarithmically transformed to better visualize large ranges of data. The function then creates a plot of the histogram using matplotlib, with the colormap set to 'jet'. 


# Results and Discussion
## Selecting a model architecture family

From the readily available model architecture families provided by [Pytorch Image Models](https://github.com/huggingface/pytorch-image-models#models), 68 unique ones were chosen as representative examples of a given class and filtered based on their parameter count, selecting those with parameters counts between 10 and 20 million to allow for an unbiased comparison. Out of the 68 ones selected 32 were subsequently underwent training with validation metrics recorded. According to @tbl-exp-1-result (arranged according to validation loss), the MobileOne S3 emerged as the top performer with an F1 score of 0.95. It was followed by DenseNet (F1 = 0.92), MobileViTV2 (F1 = 0.90), and ConvNeXt Nano (F1 = 0.84). RepVit M3 rounded out the top five with an F1 score of 0.86. To delve deeper into the performance of each architecture family, we evaluated all individual architectures within each family (@tbl-exp-1-best_models).

In [ ]:
# | label: tbl-exp-1-result
# | tbl-cap: Validation metrics of model architectural families from the PyTorch Image Models library using models with parameter counts ranging from 10 to 20 million.

(
    pd.read_csv("exp-1-result.csv")
    .rename(
        columns=lambda df: df.replace("_", " ").replace("val", "validation").title()
    )
    .rename(columns={"Minimal Param Model Count": "Parameter Count (M)"})
    .round(2)
)

As shown in @tbl-exp-1-best_models, the top four positions based on the validation metrics were occupied by models belonging to the ConvNeXt family. These models, particularly the first two (convnext_large_mlp.clip_laion2b_augreg_ft_in1k_384 with validation loss 0.19, convnext_large_mlp.clip_laion2b_augreg_ft_in1k with validation loss 0.22), were pre-trained on the extensive LAION-2B dataset and fine-tuned on ImageNet-1k, enabling them to learn complex patterns that generalize well to unseen data. This comprehensive dataset and model structure contribute to their superior performance in this task. The dominance of the ConvNeXt family is noteworthy, suggesting their effectiveness in handling complex data such as mass spectrometry pseudoimages. Apple's MobileOne also demonstrated remarkable results (validation loss = 0.27), ranking fourth in terms of validation loss. Finally, MobileViT (validation loss = 0.27), a lightweight network, secured the seventh position.  
Nevertheless, it is essential to acknowledge that while the ConvNeXt model achieved the highest validation metrics, it is considerably larger and more parameter-intensive. This implies that it demands more computational resources for training and necessitates greater care to prevent potential overfitting, as opposed to other models that are 10-20 times smaller.

In [ ]:
# | label: tbl-exp-1-best_models
# | tbl-cap: Top 10 models based on evaluation metrics of all model architectures in families from the PyTorch Image Models library, regardless of their parameter counts.

(
    pd.read_csv("exp-1-best_models.csv")
    .rename(
        columns=lambda df: df.replace("_", " ").replace("val", "validation").title()
    )
    .round(2)
    .head(10)
)

To further assess the suitability of these models and the consistency of their performance, we trained the top three performing models from each family five consecutive times and calculated the median and standard deviation of their validation metrics (@fig-exp-2-replicates_result). This approach allowed us to identify the models that exhibited the most consistent performance across multiple training runs. The results revealed that the MobileViT2 family consistently performed similarly to the ConvNeXt across all five training runs, except for the validation losses where ConvNeXt exhibited the lowest values with 0.21. According to the replication study, the ConvNeXt emerged as the most effective model, surpassing both MobileOne and MobileViT2. Despite exhibiting similar patterns, ConvNeXt exhibited a statistically significant advantage over MobileViT2 in validation loss (p = 0.01) and achieved significantly better validation recall (p < 0.05). Among the three models tested, MobileOne consistently underperformed its counterparts in all performance metrics, except for validation recall, where it narrowly outperformed MobileViT2 (p < 0.05). In light of these findings, we opted to use the ConvNeXt large model for our research.


In [ ]:
# | label: fig-exp-2-replicates_result
# | fig-cap: "Median and standard deviation of best validation metrics achieved during consecutive trainings"

Image.open("exp-2-replicates_result.png").convert("RGB")

## Selecting image properties as hyperparameters

Next, we explored the impact of image sharpness and offsets along the x and y axis as a method for image augmentation to replicate the effects of Rt shifts and incidental mass calibration drifts. @fig-exp-3-image_example illustrates how the number of bins employed in the `np.histogram2d` function within the `plot_2D_spectra_overview` function—which serves to transform LC/MS files into pseudo images—influences the image sharpness. Here, the bins parameter signifies the number of bins in both the x and y dimensions, which are represented by nx and ny, respectively. This results in the formation of an nx by ny bin grid across the data for the histogram. By increasing the number of bins (from 500, 500 to 1000, 1000), one could obtain a sharper image, but it could also lead to increased noise and computational complexity. 

Furthermore, we sought to evaluate the impact of image augmentation, specifically the generation of multiple, modified images prior to image training, on the validation metrics. The `augment_images` function applies random offsets to the specified image, producing a designated number— 9 in our case—of augmented images with a maximum horizontal offset of five and a maximum vertical offset of five pixels. 

In [ ]:
# | label: fig-exp-3-image_example
# | fig-cap: "Examples of images obtained after applying different bin sizes and offsets as imitation of Rt shift. The amount of offset along the x and y axes is given in the image name. Image quality as a function of the number of bins is set to either 500, indicating a lower-quality image, or 1000, indicating a higher-quality image."

img_paths = list(Path.cwd().glob("exp_3*"))

fig = plt.figure(figsize=(9, 9))
rows, cols = 2, 2

for idx, img_path in zip(range(1, rows * cols + 1), img_paths):
    fig.add_subplot(rows, cols, idx)
    img = Image.open(img_path).convert("RGB")
    plt.imshow(img)
    plt.title(img_path.stem[6:])
    plt.axis(False)



@fig-exp_3_experiment_result_1e 2 illustrates, image sharpness in combination with image augmentation had a significant impact on the validation loss. Setting the bin size to 500 and subsequent augmentation resulted in the lowest validation loss (0.129) and the highest validation F1 (0.982). Based on these findings, less image sharpness and more training images, obtained through data augmentation prior to training, had a beneficial effect on the validation metrics. Taken together, the final training and fine-tuning should be conducted on images with a bin size set to 500 and subsequent augmentation.

In [ ]:
# | label: fig-exp_3_experiment_result_1
# | fig-cap: "Training and validation metrics of experiments involving the Selection of image properties as hyperparameters"

Image.open("exp_3_experiment_result_1.png").convert("RGB")

## Hyperparameter tuning with Optuna

Optuna, an open-source hyperparameter optimization framework, delivers an automated and efficient approach for optimizing hyperparameters in machine learning and deep learning models. The framework operates by defining an objective function that needs to be maximized or minimized. This function encapsulates the model and the metric to be optimized. Within the objective function, hyperparameters are suggested using a trial object. Optuna supports various types of hyperparameters, including float, integer, and categorical. A study object is created and the optimize method is invoked, specifying the number of trials. The core of Optuna's operation lies in its optimization process, which employs state-of-the-art algorithms to search the hyperparameter space efficiently. It also prunes unpromising trials to conserve computational resources. This is where Bayesian optimization plays a crucial role, guiding the search based on information from previous trials. The best hyperparameters found during the search can be obtained from the study object. Optuna also provides a real-time web dashboard for tracking the optimization process, allowing users to monitor the progress of the optimization and identify potential bottlenecks.  
To optimize the model's performance, we employed Optuna to identify the most effective optimizer from a pool of four: Adam, AdamW, Adamax, and RMSprop. Additionally, we sought the optimal learning rate scheduler, choosing between CosineAnnealingLR, based on the validation loss achieved during training. After extensive optimization, the combination of Adamax optimizer and CosineAnnealingLR learning rate scheduler yielded the lowest validation loss (0.23), solidifying its selection for subsequent experiments.

## Evaluating the final model on the test set

The final model training was conducted using the optimal settings determined during the optimization experiments, employing transfer learning of a pre-trained ConvNeXt large model, with all of its layers frozen except for the last classification head. The learning rate was automatically determined to be 0.006 based on the learning rate finder tool developed by PyTorch Lightning. For the training, we utilized the augmented dataset (fig-exp-5-transformed_grid) that featured bin size set to 500 for both dimensions. To prevent overfitting and promote better generalization capabilities, random erasing was employed with a probability of 1, and color jitter was also utilized (probability=0.25). This technique effectively augments the training data, prompting the model to prioritize more essential features and minimize its dependence on specific regions of the image. This prevents the model from memorizing limited patterns in the training data and encourages it to generalize better to unseen data. The underlying concept resembles a dropout layer, a common component of neural network architectures. The optimized model achieved validation metrics with loss of 0.138, validation precision of 0.963, F1 0.946, and accuracy exceeding 0.944, and validation recall reaching 0.93.

During training, we employed early stopping to prevent overfitting, terminating training if the validation loss fails to improve for 10 consecutive epochs. Additionally, we utilized PyTorch Lightning's built-in ModelCheckpoint to save the best-performing model, ensuring that the model that achieved the highest validation metrics was preserved.

In [ ]:
# | label: fig-exp-5-transformed_grid
# | fig-cap: "Example of augmented dataloader images used during training time, demonstrating the application of random erasing and color jitter to reduce overfitting"

Image.open("exp-5-transformed_grid.png").convert("RGB")

After the training was completed, the best-performing model was reloaded, and test metrics were calculated using the 30 test samples present in our dataset. According to the final evaluation, the model produced 20 True Positives, 7 True Negatives, 2 False Negatives, and 1 False Positive. These results translate to a precision of 0.952, recall 0.909, F1 0.930, and accuracy 0.90. While the model achieved impressive validation metrics, the slight drop in performance on the test set indicates a potential overfitting issue. Further optimization of the model, particularly during the method optimization phase, could be achieved through cross-validation strategies. However, this approach might necessitate excessive computational resources.

## Model interpretability 

To assess the model's confidence in its predictions, we can transform the raw output, which is the binary cross-entropy loss derived from a set of logits obtained using PyTorch's BCEWithLogitsLoss function, into prediction probabilities using a sigmoid function. These converted values reflect the probability that the model assigns to a given prediction belonging to Class 1 (Opioid User). @fig-exp-5-prediction_matrix illustrates this concept, showing that in the first instance, the model estimates a 43% probability of the instance belonging to Class 1, suggesting a higher likelihood of it being a Non-Opioid User (and this prediction proved correct).

In [ ]:
# | label: fig-exp-5-prediction_matrix
# | fig-cap: "Set of images from the test set, along with their corresponding prediction probabilities for the Class 1 (Opidid User). The true labels are displayed beside the predictions. Green labels indicate successful predictions, whereas red labels denote unsuccessful ones"

Image.open("exp-5-prediction_matrix.png").convert("RGB")

To gain insights into the regions of an image our model focuses on when making a prediction, we employed LayerCAM, a tool from the TorchCam package. LayerCAM (@jiang_layercam_2021), short for Layer Class Activation Maps, is a technique for generating class activation maps from distinct layers of a CNN. These regions of interests highlight the key features that the model utilizes to differentiate between opioid users and non-opioid users. Class activation maps are typically derived from the final convolutional layer of a CNN, highlighting discriminative object regions for the class of interest. Such a tool would be particularly valuable for validating the insights derived from our deep learning model, particularly when compared to more targeted or untargeted metabolomics data. 

In [ ]:
# | label: fig-exp-5-plot_activation
# | fig-cap: "Regions of interest identified by LayerCAM, demonstrating the specific areas of the image that our model deems most relevant for making predictions."

Image.open("exp-5-plot_activation.png").convert("RGB")

In [ ]:
import io
import re

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from PIL import Image


def overlay_untargeted_data(
    query_str, ms_dial_location, background_image_location, crop_area
):
    """
    Overlay untargeted metabolomics data on a background image.

    Parameters:
    - query_str (str): Query string for filtering the data.
    - ms_dial_location (str): Location of the mass spectrometry data file.
    - background_image_location (str): Location of the background image.
    - crop_area (tuple): Coordinates for cropping the background image (left, upper, right, lower).

    Returns:
    - composite_img (PIL.Image.Image): Overlay of data on the background image.
    - df (pd.DataFrame): Filtered and processed DataFrame.
    """
    # Read and preprocess the mass spectrometry data
    df = (
        pd.read_csv(
            ms_dial_location,
            skiprows=4,
            usecols=[
                "Average Rt(min)",
                "Average Mz",
                "Metabolite name",
                "Ontology",
                "Annotation tag (VS1.0)",
                "MS/MS matched",
                "SP_10",
                "SP_20",
            ],
        )
        .rename(columns=lambda x: re.sub(r"\W|[%/]", "_", x.lower()))
        .query("annotation_tag__vs1_0_ == '430' or annotation_tag__vs1_0_ == '530'")
        .query(query_str)
        .drop_duplicates(subset="metabolite_name")
    )

    # Create scatterplot
    sns.scatterplot(
        data=df,
        x="average_rt_min_",
        y="average_mz",
        alpha=0.5,
        s=100,
        color="black",
    )

    # Set plot limits and turn off axis
    plt.ylim(50, 750)
    plt.xlim(0, 20)
    plt.axis(False)

    # Create the foreground image
    buf = io.BytesIO()
    plt.savefig(buf, format="jpeg", bbox_inches="tight", pad_inches=0)
    buf.seek(0)
    foreground = Image.open(buf)
    plt.close()

    # Create the background image
    background = (
        Image.open(background_image_location)
        .convert("RGB")
        .crop(crop_area)
        .resize(foreground.size)
    )

    # Create a mask
    mask = Image.new("L", foreground.size, 128)

    # Composite the images
    composite_img = Image.composite(background, foreground, mask)

    return composite_img, df

In [ ]:
composite_img_pattern_1, pattern_df_1 = overlay_untargeted_data(
    query_str="3.5<average_rt_min_<5 and 460<average_mz<500 or 5<average_rt_min_<6.5 and 575<average_mz<650 or 9.5<average_rt_min_<10 and 280<average_mz<320",
    ms_dial_location="PeakID_0_2023_12_25_17_39_09.csv",
    background_image_location="exp-5-plot_activation.png",
    crop_area=(25, 90, 850, 910),
)

composite_img_pattern_1

In [ ]:
composite_img_pattern_2, pattern_df_2 = overlay_untargeted_data(
    query_str="(16.5<average_rt_min_<18 and 180<average_mz<250) or (4<average_rt_min_<5.5 and 50<average_mz<100) or (8<average_rt_min_<12 and 50<average_mz<100) or (8.5<average_rt_min_<9 and 250<average_mz<300) or (12<average_rt_min_<14 and 200<average_mz<220) or (18<average_rt_min_<20 and 280<average_mz<350)",
    ms_dial_location="PeakID_0_2023_12_25_17_39_09.csv",
    background_image_location="exp-5-plot_activation.png",
    crop_area=(20, 1070, 850, 1890),
)

composite_img_pattern_2

In [ ]:
pd.concat([pattern_df_1, pattern_df_2], axis=0).query("annotation_tag__vs1_0_ == '430'")

# Get in touch

Did the app help with your research? Any ideas for making it better? Get in touch! I would love to hear from you.